This NoteBook tests out various card transformations that can be implemented. In addition it also tests various ways of stroing this spatial data
in a PostGIS DB

Python Frameworks used:
- geoalchemy2
- shapely
- geopandas

In [1]:
"""
For setting up local imports in an Ipython Shell
This is a workaround for ipython, dont need it for basic python scripts
"""
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import pandas as pd
import numpy as np
from library.dbconnection import PostgresRDS
from library.sqlFunctions import run_query
from library.cardFunctions import Cleaning

In [5]:
"""
Import a test data from the db
"""
well_name = 'A. JOHNSON 12-1H'

query = """
SELECT 
"NodeID","Date",encode("tblCardData"."DownholeCardB", 'hex') as downcard
FROM xspoc_dbo."tblCardData"
WHERE "NodeID" = '{}'
ORDER BY "NodeID" , "Date";
""".format(well_name)

with PostgresRDS(db='oasis-data') as engine:
    well_data = run_query(query, engine)
    
display(well_data.head())

Connected to oasis-data DataBase
Connection Closed


,NodeID,Date,downcard
0,A. JOHNSON 12-1H,2019-03-19 01:38:50,00fe654280eef1410030e1400000a23d80aba54140c19b...
1,A. JOHNSON 12-1H,2019-03-20 01:41:44,e0519443686e8d4348548243e0e07043b0626b4310867f...
2,A. JOHNSON 12-1H,2019-03-20 05:23:25,d027564310b04e43100e3643806416432052f8424020e9...
3,A. JOHNSON 12-1H,2019-03-20 07:27:21,0027e04200d1bc4280867f424051004200e24041803a85...
4,A. JOHNSON 12-1H,2019-03-20 09:25:37,c05b7a4240c50a42002c6640007a8ec1005184c1001096...


### Method 1:

This the method used currently
- Convert hex values into a wkb format
    - Hex converted into xy array (`cardFunctions.Cleaning.get_dyna()`)
    - xy converted into a shapely polygon (`shapely.geometry.Polygon()`) 
    - srid is specified (4326) (`geoalchemy2.shape.from_shape()`)
- This wkb format cards can be added to a PostGIS database.
    - Specify the `dtype` in `pd.to_sql()` as `{'card_col': 'Geometry("POLYGON")'}`
    - Note: This is from `geoalchmey2.Geometry` and dont specify an 'srid'

### Method 2
*Works*
Steps:
- Convert hex to a shapely Polygon.
- Convert the pandas df into a GeoDataFrame.
- Set the crs value using `df.crs = "EPSG:4326"`.
- Convert POLYGON geometry into a WKTElement.
- Can be added to the postgis db as in method 1.
- Specifying srid in the addtion method raises an error. (Find out why)


In [8]:
from geoalchemy2.elements import WKBElement, WKTElement
from geopandas import GeoDataFrame
from shapely.geometry import Polygon

# FUnction to convert hex --> poly
def hex_to_poly(card):
    xy = Cleaning.get_dyna(card)
    try:
        poly= Polygon(xy)
    except Exception as e:
        print(e)
        poly=np.nan
    
    return poly

# Use GeoAlchemy's WKTElement to create a geom with SRID
def create_wkt(geom):
    return WKTElement(geom.wkt, srid = 4326)

In [11]:
df2 = well_data.copy()
df2.downcard = df2.downcard.apply(hex_to_poly)  # Comvert from hex --> poly
df2_geo = GeoDataFrame(df2)  # Convert to geodataframe
df2_geo.crs = "EPSG:4326"
df2_geo.downcard = df2_geo.downcard.apply(create_wkt)
df2_geo.head()

### Pulling Data

While pulling data from the PostGIS db, or when the wkb format is in a string form (will not have a `<WKBElement>` superscript), Techniques shown below can be used.

In [14]:
# Pull data from a PostGIS DB (oasis-dev)
# Table name is 'card'
well_name = 'Bonner 9X-12HA'

query = """
SELECT 
"NodeID","Date", downcard  -- Dont have to encode or decode
FROM card
WHERE "NodeID" = '{}'
ORDER BY "NodeID" , "Date";
""".format(well_name)

with PostgresRDS(db='oasis-dev') as engine:
    well_data = run_query(query, engine)
    
display(well_data.head())
display(well_data.dtypes)

Connected to oasis-dev DataBase
Connection Closed


,NodeID,Date,downcard
0,Bonner 9X-12HA,2019-03-19 01:30:10,01030000000100000064000000000000A070BD25400000...
1,Bonner 9X-12HA,2019-03-19 01:50:25,01030000000100000064000000000000803D8A25400000...
2,Bonner 9X-12HA,2019-03-19 02:17:17,010300000001000000640000000000000029DC31400000...
3,Bonner 9X-12HA,2019-03-19 05:12:56,010300000001000000640000000000002085EB2F400000...
4,Bonner 9X-12HA,2019-03-19 05:58:01,0103000000010000006400000000000000000015400000...


In [30]:
"""
Can use the loads method from shapely.wkb to get the polygon obj from a str based wkb object
"""
from shapely.wkb import loads, dumps

poly_series = well_data.loc[:, 'downcard'].apply(lambda card: loads(card, hex=True)) 
display(poly_series.head())

0    POLYGON ((10.86999988555908 -8382, 10.64000034...
1    POLYGON ((10.77000045776367 -7438, 29.34000015...
2    POLYGON ((17.86000061035156 -8063, 22.13999938...
3    POLYGON ((15.96000003814697 -8104, 18.61000061...
4    POLYGON ((5.25 -6926, 17.53000068664551 -8199,...
Name: downcard, dtype: object

In [28]:
"""
Can Convert it into a WKTElement from the polygon with an SRID
using the method WKTElement from geoalchemy2.elements
"""
poly_wkt = poly_series.apply(create_wkt)

display(poly_wkt.head())

# NOTE: Can perform all this with a GeoDataFrame as well
# Find out what might be the resons for doing it like that

0    POLYGON ((10.86999988555908 -8382, 10.64000034...
1    POLYGON ((10.77000045776367 -7438, 29.34000015...
2    POLYGON ((17.86000061035156 -8063, 22.13999938...
3    POLYGON ((15.96000003814697 -8104, 18.61000061...
4    POLYGON ((5.25 -6926, 17.53000068664551 -8199,...
Name: downcard, dtype: object

### Converting to WKBElement
From either a polygon obj or a str type obj

&nbsp;

**Method1: dumps from shapely.wkb**
- Dumps will convert it into str obj type wkb
- May not be helpful
- Has previously shown to cause errors will adding it back to postgis

&nbsp;

**Method2: using the .wkb_hex extension**
- Does the same thing as dumps
- Similarly needs to be further processed



**Method3: Converting these str type wkb objects to WKBElements**
- Can use 'WKBElement' from geoalchemy2.elements
- Can specify an srid
- Will give the 'WKBElement' prefix
- To convert directly from poly to WKBElement can use it in conjuction with Method2.

```
poly_series.apply(lambda poly: WKBElement(poly.wkb, srid='4326'))[0]
# or
poly_series.apply(lambda poly: WKBElement(poly.wkb_hex, srid='4326'))[0]
```

&nbsp;

**NOTE:**
- Converting WKBElemnt to polygon shape cannot be done using `shapely.wkb.loads()`.
- We have to use `geoalchemy2.shape.to_shape()`

In [51]:
"""
Method1
"""
print("Only showing first 50 elements")
wkb_from_dump = poly_series.apply(lambda poly: dumps(poly, hex=True))
print("\nMethod1: polygon to wkb From dumps")
print(wkb_from_dump[0][0:50])

wkb_method2 = poly_series.apply(lambda poly: poly.wkb_hex)
print("\nMethod2: polygon to wkb from .wkb_hex")
print(wkb_method2[0][0:50])

print('\nOriginal wkb, pulled from PostGIS DB')
print(well_data.downcard[0][0:50])

print("\n-------------")
print("Any of this can be converted into a WKBElement using method3")
wkb_element = well_data.downcard.apply(lambda x: WKBElement(x, srid='4326'))
print(wkb_element[0])


Only showing first 50 elements

Method1: polygon to wkb From dumps
01030000000100000064000000000000A070BD254000000000

Method2: polygon to wkb from .wkb_hex
01030000000100000064000000000000A070BD254000000000

Original wkb, pulled from PostGIS DB
01030000000100000064000000000000A070BD254000000000

-------------
Any of this can be converted into a WKBElement using method3
01030000000100000064000000000000A070BD254000000000005FC0C000000020AE47254000000000800FC1C000000080C2F52340000000000051C1C0000000A099192340000000000085C1C000000000D7A323400000000080AFC1C000000020856B26400000000080D0C1C0000000803D0A2C400000000080E4C1C000000020856B32400000000000E7C1C000000060666638400000000080D4C1C0000000C0CCCC3F400000000080B0C1C000000000D7234440000000000082C1C0000000803DAA4840000000008052C1C000000080142E4D40000000008029C1C0000000608FB2504000000000000CC1C0000000E0518852400000000000FBC0C000000080C20554400000000000F1C0C000000080C22555400000000000E3C0C00000008014EE55400000000080BEC0C000000080146E5640000000008

In [69]:
"""
Connverting WKBElement to Polygon
"""
from geoalchemy2.shape import to_shape
print("Polygon Obj from WKBElement\n")
print(wkb_element.apply(lambda x: to_shape(x)).head())

Polygon Obj from WKBElement

0    POLYGON ((10.86999988555908 -8382, 10.64000034...
1    POLYGON ((10.77000045776367 -7438, 29.34000015...
2    POLYGON ((17.86000061035156 -8063, 22.13999938...
3    POLYGON ((15.96000003814697 -8104, 18.61000061...
4    POLYGON ((5.25 -6926, 17.53000068664551 -8199,...
Name: downcard, dtype: object
